In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Configuración Inicial

En esta celda, cargamos las variables de entorno desde un archivo `.env` (si existe). Esto permite configurar el modelo de Ollama y la URL del servidor de forma flexible, sin hardcodear valores en el código.

- Usamos `load_dotenv()` para leer el archivo `.env`.
- Definimos `MODEL` como el modelo de Ollama a usar (por defecto `llama3.1:8b` para mejor razonamiento).
- `BASE_URL` apunta al servidor local de Ollama (por defecto `http://localhost:11434`).

Luego, inicializamos el LLM con `ChatOllama`, que es una interfaz open source para modelos locales. Esto evita depender de APIs propietarias.

In [2]:
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv

load_dotenv()
MODEL = os.getenv("MODEL", "llama3.1:8b")
BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")

llm = ChatOllama(model=MODEL, base_url=BASE_URL, temperature=0)
response = llm.invoke('Hola soy Nicolas y mi telefono es +57 317 867 2615')
response.content

'¡Hola Nicolas! Encantado de conocerte. ¿En qué puedo ayudarte hoy? Si tienes alguna pregunta o necesitas asistencia con algo en particular, no dudes en decírmelo.'

## Invocación Básica del LLM

Aquí invocamos el modelo de Ollama con un mensaje simple. El modelo responde de forma conversacional, procesando el input y devolviendo una respuesta en texto plano.

- Usamos `ChatOllama` para conectar con el modelo local.
- `invoke()` envía el mensaje y devuelve un objeto con `content` (la respuesta).
- Esto demuestra el uso básico de LLMs locales, sin necesidad de APIs externas.

Nota: La respuesta es en español, ya que el modelo (llama3.1:8b) es capaz de manejar múltiples idiomas.

In [3]:
from pydantic import BaseModel, Field
from typing import Optional

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: Optional[str] = Field(description="The name of the person, if mentioned explicitly.")
    email: Optional[str] = Field(description="The email address of the person, if provided.")
    phone: Optional[str] = Field(description="The phone number of the person, if given.")
    tone: Optional[int] = Field(description="The tone of the person on a scale from 0 to 100, if inferable.", ge=0, le=100)
    age: Optional[int] = Field(description="The age of the person, if mentioned as a number.")
    sentiment: Optional[str] = Field(description="The sentiment of the conversation: positive, negative, or neutral, if inferable.")

llm_with_structured_output  = llm.with_structured_output(schema=ContactInfo)

messages = [
    ("system", "Extrae información de contacto de la conversación. Si no encuentras un dato explícitamente, no lo inventes. Para sentimiento, infiere basado en el tono del mensaje: positivo, negativo o neutral. Usa null solo si no se puede inferir."),
    ("user", "Hola soy Nicolas y mi telefono es +57 317 867 2615"),
]

response = llm_with_structured_output.invoke(messages)
response.name

'Nicolas'

## Salida Estructurada con Pydantic

En esta sección, usamos `with_structured_output()` para forzar al LLM a devolver datos en un formato específico definido por un esquema Pydantic.

- Definimos `ContactInfo` como un modelo Pydantic con campos opcionales (usando `Optional` para evitar errores si no se encuentra el dato).
- El esquema incluye descripciones claras para guiar al modelo.
- El prompt del sistema instruye al modelo a extraer información y, para sentimiento, inferir basado en el tono (positivo, negativo o neutral).
- La respuesta es un objeto Pydantic, que podemos acceder como atributos (ej. `response.name`).

Esto transforma respuestas impredecibles en datos estructurados, útiles para agentes que necesitan información confiable.

In [4]:
f"{response}"

"name='Nicolas' email=None phone='+57 317 867 2615' tone=None age=None sentiment=None"

## Análisis del Resultado

El resultado muestra los datos extraídos:
- `name='Nicolas'`: Extraído directamente del mensaje.
- `phone='+57 317 867 2615'`: Teléfono proporcionado.
- Otros campos como `email`, `tone`, `age` y `sentiment` son `None` porque no se mencionan explícitamente o no se pueden inferir con confianza.

Esto demuestra cómo el structured output evita alucinaciones: el modelo no inventa datos que no están presentes. Para mejorar la inferencia de sentimiento, podríamos ajustar el prompt o usar un modelo más avanzado, pero este enfoque prioriza precisión sobre suposiciones.